## ORBIT


### Getting started:


0. Prereqs: Install cairo library
On linux:
`sudo apt-get install libcairo2-dev`

1. Create a virtualenv:
`virtualenv --python=python3 ~/venv/py36`

2. Activate virtualenv:
`source ~/venv/py36/bin/activate`

3. Install packages
`pip install imageio pycairo`



In [1]:
import imageio
import cairo 
import math
import tempfile
import os


FRAME_COUNT = 150
WIDTH = 800
HEIGHT = 800

In [2]:

class CelestialBody:
    def __init__(self, radius, orbit_radius, orbit_center, fill, parent=None, pc=0, col=(1, 1, 1)):
        self.radius = radius
        self.orbit_radius = orbit_radius
        self.orbit_center = orbit_center
        self.parent_body = parent
        self.phase = 0.0
        self.phase_const = pc
        self.fill = fill
        self.col = col
        self.update()

    def update(self):
        if self.parent_body is not None:
            self.orbit_center = self.parent_body.center

        self.center = (
            self.orbit_center[0] + self.orbit_radius * math.cos(self.phase),
            self.orbit_center[1] + self.orbit_radius * math.sin(self.phase))
        
    def set_phase(self, i):
        self.phase = i * self.phase_const * math.pi / FRAME_COUNT
        self.update()

    def draw(self, context):
        context.set_source_rgb(*self.col)
        context.arc(*self.center, self.radius, 0, 2 * math.pi)
        if self.fill:
            context.fill()
        else:
            context.stroke()

def clear_screen(context):
    context.set_source_rgb(0, 0, 0)
    context.rectangle(0, 0, WIDTH, HEIGHT)
    context.fill()


In [3]:
sun = CelestialBody(70, 0, (WIDTH//2, HEIGHT//2), True, None, 0, (1,1,1))

planets = {
    'earth' : CelestialBody(15,  200, (750, 500), True, sun, 3, (0.5,0.5,0.9)),
    'jupiter' : CelestialBody(30,  300, (300, 300), True, sun, 1, (0.9,0.3,0.3))
}

moons = {
    'moon':  CelestialBody(3, 25,  (750, 500), False, planets['earth'], 7, (1,1,1)) 
}

bodies = {
    'sun' : sun,
    **planets, 
    **moons
}

In [4]:
images = []
surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, WIDTH, HEIGHT)
context = cairo.Context(surface)



for i in range(0, FRAME_COUNT):
    clear_screen(context)

    for _, body in bodies.items():
        body.set_phase(i)
        body.draw(context)

    context.stroke()
    temp_png = f"/tmp/{i}.png".format(i)
    surface.write_to_png(temp_png)
    images.append(imageio.imread(temp_png))
    os.remove(temp_png)


imageio.mimsave('planets.gif', images)

In [6]:
# jupyter notebook only...

from IPython.core.display import HTML, Image
Image(url="planets.gif", width=800, height=800)